In [1]:
# 读取模拟数据
# 读取模拟数据
import pandas as pd
df = pd.read_csv('data_simulation/data_generated/data_simulate_04.csv')
date_columns = ["财务数据日期", "公告日期", "日期"]
for col in date_columns:
    df[col] = pd.to_datetime(df[col])
df

FileNotFoundError: [Errno 2] No such file or directory: 'data_simulation/data_generated/data_simulate_04.csv'

# Law Article 4

## MEU_4_1


| 字段 | 内容 |
|------|------|
| subject | 上市公司 |
| condition | 因维护公司价值及股东权益所必需回购股份的 |
| constraint | 应当符合以下条件之一：（一）公司股票收盘价格低于最近一期每股净资产；（二）连续20个交易日内公司股票收盘价格跌幅累计达到20%；（三）公司股票收盘价格低于最近一年股票最高收盘价格的50%；（四）中国证券监督管理委员会（以下简称中国证监会）规定的其他条件。 |
| contextual_info | nan |
| note | nan |
| relation | nan |
| target | nan |
| type | 实际执行单元 |
| comments | nan |
| prompt_tokens | 1855 |
| completion_tokens | 4160 |


### 代码实现

In [ ]:
import pandas as pd

def check_meu_4_1(df):
    '''
    检查MEU_4_1合规性：
    - subject: 上市公司（所有记录自动满足）
    - condition: 存在维护公司价值的回购方案
    - constraint: 需满足四个价格条件之一（仅验证前三项）
    '''
    df = df.copy()

    # 初始化合规标记列
    df['meu_4_1_subject'] = True  # 所有记录均为上市公司
    df['meu_4_1_condition'] = False
    df['meu_4_1_constraint'] = None

    # 标记condition条件（存在特定目的的回购方案）
    valid_condition = df['存在回购方案'] & (df['回购用途'] == '维护公司价值及股东权益所必需')
    df['meu_4_1_condition'] = valid_condition

    # 计算constraint的三个可验证条件
    # 条件1：收盘价低于每股净资产
    condition1 = df['收盘价'] < df['每股净资产']

    # 条件2：20交易日累计跌幅20%（按公司分组计算）
    df['20d_prior_close'] = df.groupby('公司简称')['收盘价'].shift(20)
    price_drop = (df['收盘价'] - df['20d_prior_close']) / df['20d_prior_close']
    condition2 = price_drop <= -0.2

    # 条件3：低于一年最高价50%（滚动250交易日窗口）
    df['1y_high'] = df.groupby('公司简称')['收盘价'].transform(
        lambda x: x.rolling(250, min_periods=1).max()
    )
    condition3 = df['收盘价'] < (df['1y_high'] * 0.5)

    # 合并有效约束条件（前三项）
    valid_constraint = condition1 | condition2 | condition3
    
    # 标记constraint列
    df['meu_4_1_constraint'] = valid_constraint.astype(bool)
    
    # 清理中间计算列
    df.drop(['20d_prior_close', '1y_high'], axis=1, inplace=True, errors='ignore')

    return df

In [ ]:
df = check_meu_4_1(df)
df

---

# Law Article 13

## MEU_13_1


| 字段 | 内容 |
|------|------|
| subject | 上市公司 |
| condition | 实施竞价回购 |
| constraint | 应当符合公司股票上市已满6个月 |
| contextual_info | nan |
| note | 假设公司股票上市日为最早的收盘价非空的日期 |
| relation | nan |
| target | nan |
| type | 实际执行单元 |
| comments | nan |
| prompt_tokens | 1800 |
| completion_tokens | 1896 |


### 代码实现

In [ ]:
import pandas as pd

def check_meu_13_1(df):
    '''
    验证MEU_13_1合规性:
    - subject: 上市公司
    - condition: 实施竞价回购
    - constraint: 上市已满6个月
    
    参数说明:
    df: 包含公司每日状态的面板数据，必须包含['上市日期','日期','回购方式']列
    
    验证逻辑:
    1. 所有公司默认满足subject条件（均为上市公司）
    2. 当回购方式为竞价交易时满足condition
    3. 当当前日期距离上市日期满6个月时满足constraint
    '''
    
    df = df.copy()
    
    # 初始化标记列
    df['meu_13_1_subject'] = True  # 所有公司均为上市公司
    df['meu_13_1_condition'] = False
    df['meu_13_1_constraint'] = False
    
    # 标记condition条件（实施竞价回购）
    is_auction_repurchase = df['回购方式'] == '竞价回购'
    df.loc[is_auction_repurchase, 'meu_13_1_condition'] = True
    
    # 标记constraint条件（上市满6个月）
    # 计算上市日期+6个月后的日期
    six_months_after = df['上市日期'] + pd.DateOffset(months=6)
    # 比较当前日期是否满足时间要求（包含等于）
    valid_constraint = df['日期'] >= six_months_after
    df.loc[valid_constraint, 'meu_13_1_constraint'] = True
    
    return df

In [ ]:
df = check_meu_13_1(df)
df

---

## MEU_13_2


| 字段 | 内容 |
|------|------|
| subject | 上市公司 |
| condition | 实施竞价回购 |
| constraint | 应当符合公司最近1年无重大违法行为 |
| contextual_info | nan |
| note | nan |
| relation | nan |
| target | nan |
| type | 实际执行单元 |
| comments | nan |
| prompt_tokens | 1789 |
| completion_tokens | 4860 |


### 代码实现

In [ ]:
import pandas as pd

def check_meu_13_2(df):
    '''
    合规性检查函数，针对MEU_13_2条款：
    - subject: 上市公司
    - condition: 实施竞价回购
    - constraint: 最近1年无重大违法行为
    
    参数说明：
    df : 包含上市公司每日状态数据的DataFrame
    
    返回：
    添加合规性标记后的DataFrame
    '''
    df = df.copy()

    # 初始化标记列
    df['meu_13_2_subject'] = True  # 所有公司均为上市公司
    df['meu_13_2_condition'] = False
    df['meu_13_2_constraint'] = None

    # 条件1: 实施竞价回购
    valid_condition = df['存在回购方案'] & (df['回购方式'] == '竞价回购')
    df.loc[valid_condition, 'meu_13_2_condition'] = True

    # 条件2: 最近1年无重大违法行为
    # 创建临时违法标记
    df['重大违法'] = df['公司法律情况'] == '重大违法行为'
    
    # 按公司分组进行时间序列计算
    df_sorted = df.sort_values(['公司简称', '日期'])
    df_sorted['重大违法_过去一年'] = df_sorted.groupby('公司简称')['重大违法'].apply(
        lambda x: x.rolling('365D', closed='left').max().fillna(False)
    )
    
    # 合并计算结果
    df = df.merge(df_sorted[['重大违法_过去一年']], left_index=True, right_index=True, how='left')
    
    # 设置约束条件标记
    df['meu_13_2_constraint'] = ~df['重大违法_过去一年']
    
    # 清理临时列
    df.drop(columns=['重大违法', '重大违法_过去一年'], inplace=True)

    return df

In [ ]:
df = check_meu_13_2(df)
df

---

## MEU_13_7


| 字段 | 内容 |
|------|------|
| subject | 上市公司 |
| condition | 因触及本指引第四条规定条件而启动回购并减少注册资本 |
| constraint | 不适用股票上市已满6个月的要求 |
| contextual_info | nan |
| note | nan |
| relation | exclude |
| target | MEU_13_1 |
| type | 实际执行单元 |
| comments | nan |
| prompt_tokens | 1799 |
| completion_tokens | 6697 |


### 代码实现

In [ ]:
import pandas as pd

def check_meu_13_7(df):
    '''
    合规性检查函数，用于验证MEU_13_7条款的遵守情况
    subject: 上市公司
    condition: 因触及本指引第四条规定条件而启动回购并减少注册资本
    constraint: 不适用股票上市已满6个月的要求
    '''
    df = df.copy()

    # 初始化标记列
    df['meu_13_7_subject'] = True  # 所有公司均为上市公司
    df['meu_13_7_condition'] = False
    df['meu_13_7_constraint'] = False

    # 条件验证：触发第四条回购情形（假设第四条对应维护公司价值的回购）
    # 1.存在有效回购方案 2.回购用途为维护公司价值 3.假设该情形对应减少注册资本
    valid_condition = (
        df['存在回购方案'] & 
        (df['回购用途'] == '维护公司价值及股东权益所必需')
    )
    df.loc[valid_condition, 'meu_13_7_condition'] = True

    # 约束验证：上市未满6个月（精确到自然月计算）
    # 计算上市日期6个月后的日期
    six_months_later = df['上市日期'] + pd.DateOffset(months=6)
    # 判断当前日期是否在6个月期限内
    df['meu_13_7_constraint'] = df['日期'] < six_months_later

    return df

In [ ]:
df = check_meu_13_7(df)
df

---

## MEU_13_7


| 字段 | 内容 |
|------|------|
| subject | 上市公司 |
| condition | 因触及本指引第四条规定条件而启动回购并减少注册资本 |
| constraint | 不适用股票上市已满6个月的要求 |
| contextual_info | nan |
| note | nan |
| relation | refer_to |
| target | MEU_13_1 |
| type | nan |
| comments | nan |
| prompt_tokens | 1799 |
| completion_tokens | 6697 |


### 代码实现

In [ ]:
import pandas as pd

def check_meu_13_7(df):
    '''
    合规性检查函数，用于验证MEU_13_7条款的遵守情况
    subject: 上市公司
    condition: 因触及本指引第四条规定条件而启动回购并减少注册资本
    constraint: 不适用股票上市已满6个月的要求
    '''
    df = df.copy()

    # 初始化标记列
    df['meu_13_7_subject'] = True  # 所有公司均为上市公司
    df['meu_13_7_condition'] = False
    df['meu_13_7_constraint'] = False

    # 条件验证：触发第四条回购情形（假设第四条对应维护公司价值的回购）
    # 1.存在有效回购方案 2.回购用途为维护公司价值 3.假设该情形对应减少注册资本
    valid_condition = (
        df['存在回购方案'] & 
        (df['回购用途'] == '维护公司价值及股东权益所必需')
    )
    df.loc[valid_condition, 'meu_13_7_condition'] = True

    # 约束验证：上市未满6个月（精确到自然月计算）
    # 计算上市日期6个月后的日期
    six_months_later = df['上市日期'] + pd.DateOffset(months=6)
    # 判断当前日期是否在6个月期限内
    df['meu_13_7_constraint'] = df['日期'] < six_months_later

    return df

In [ ]:
df = check_meu_13_7(df)
df

---

# Law Article 15

## MEU_15_2


| 字段 | 内容 |
|------|------|
| subject | 上市公司 |
| condition | nan |
| constraint | 在回购股份方案中明确拟回购股份数量或者资金总额的上下限，且下限不得低于上限的50% |
| contextual_info | nan |
| note | nan |
| relation | nan |
| target | nan |
| type | 实际执行单元 |
| comments | nan |
| prompt_tokens | 1802 |
| completion_tokens | 6138 |


### 代码实现

In [ ]:
import pandas as pd

def check_meu_15_2(df):
    '''
    检查MEU_15_2合规性:
    subject: 上市公司
    condition: NaN(无条件触发)
    constraint: 回购方案需明确数量/资金上下限且下限≥50%上限
    '''
    df = df.copy()

    # 初始化合规标记列
    df['meu_15_2_subject'] = True  # 所有公司均为上市公司
    df['meu_15_2_condition'] = True  # 无条件要求，全部视为满足
    df['meu_15_2_constraint'] = False  # 初始化为False

    # 标记constraint合规性
    # 不存在回购方案的行自动合规
    df.loc[~df['存在回购方案'], 'meu_15_2_constraint'] = True

    # 处理存在回购方案的行
    mask = df['存在回购方案']
    
    # 计算数量合规性 (下限≥50%上限)
    quantity_valid = (
        df['回购数量下限'] >= df['回购数量上限'] * 0.5
    ) & (
        df['回购数量上限'].notna()  # 确保已明确设置数量上限
    )
    
    # 计算资金合规性 (下限≥50%上限)
    funds_valid = (
        df['资金总额下限'] >= df['资金总额上限'] * 0.5
    ) & (
        df['资金总额上限'].notna()  # 确保已明确设置资金上限
    )
    
    # 合并合规条件
    combined_valid = quantity_valid | funds_valid
    
    # 更新存在回购方案行的标记
    df.loc[mask, 'meu_15_2_constraint'] = combined_valid[mask]

    return df

In [ ]:
df = check_meu_15_2(df)
df

---

# Law Article 16

## MEU_16_1


| 字段 | 内容 |
|------|------|
| subject | 上市公司 |
| condition | 进行竞价回购 |
| constraint | 价格上限原则上不应高于董事会审议通过回购股份决议前30个交易日（不含停牌日）交易均价的200% |
| contextual_info | nan |
| note | 不考虑"原则上"的松弛条件 |
| relation | refer_to |
| target | MEU_75_1 |
| type | 实际执行单元 |
| comments | nan |
| prompt_tokens | 1877 |
| completion_tokens | 7910 |


### 代码实现

In [ ]:
import pandas as pd

def check_meu_16_1(df):
    '''
    检查MEU_16_1合规性：
    - subject: 上市公司（所有行自动满足）
    - condition: 存在竞价回购行为
    - constraint: 申报价格不超过决议前30日交易均价200%
    注：因缺少成交明细数据，使用收盘价均值替代交易均价计算
    '''
    df = df.copy()
    
    # 1. 标记subject（所有公司均为上市公司）
    df['meu_16_1_subject'] = True
    
    # 2. 标记condition（存在回购方案且为竞价方式）
    df['meu_16_1_condition'] = df['存在回购方案'] & (df['回购方式'] == '竞价回购')
    
    # 3. 标记constraint（独立检查所有行）
    df['meu_16_1_constraint'] = False
    
    # 仅处理有决议日期和申报价格的行
    valid_mask = df['决议通过日'].notna() & df['申报价格'].notna()
    
    for idx in df[valid_mask].index:
        row = df.loc[idx]
        company_data = df[df['公司简称'] == row['公司简称']]
        
        # 获取决议日前30个交易日
        pre_dates = company_data[company_data['日期'] < row['决议通过日']]
        if len(pre_dates) >= 30:
            # 使用收盘价计算均价（实际应使用成交额/成交量）
            avg_price = pre_dates.nlargest(30, '日期')['收盘价'].mean()
            price_limit = avg_price * 2
            df.loc[idx, 'meu_16_1_constraint'] = row['申报价格'] <= price_limit

    return df

In [ ]:
df = check_meu_16_1(df)
df

---

# Law Article 18

## MEU_18_7


| 字段 | 内容 |
|------|------|
| subject | 上市公司 |
| condition | 进行股份回购的申报 |
| constraint | 申报价格不得为公司股票当日交易涨幅限制的价格 |
| contextual_info | nan |
| note | 假设当日涨跌幅限制为前日收盘价的正负10% |
| relation | nan |
| target | nan |
| type | 实际执行单元 |
| comments | nan |
| prompt_tokens | 1806 |
| completion_tokens | 5234 |


### 代码实现

In [ ]:
import pandas as pd

def check_meu_18_7(df):
    '''
    验证MEU_18_7合规性:
    - subject: 上市公司
    - condition: 进行股份回购的申报
    - constraint: 申报价格不得为当日涨跌幅限制价格
    '''
    df = df.copy()

    # 初始化合规性标记列
    df['meu_18_7_subject'] = True  # 所有公司均为上市公司
    df['meu_18_7_condition'] = False
    df['meu_18_7_constraint'] = False

    # 1. 标记condition条件满足情况
    # 条件成立需同时满足：存在回购方案且存在申报价格
    has_repo_plan = df['存在回购方案']
    has_declaration = df['申报价格'].notna()
    valid_condition = has_repo_plan & has_declaration
    df.loc[valid_condition, 'meu_18_7_condition'] = True

    # 2. 标记constraint约束满足情况
    # 计算前收盘价的±10%涨跌幅限制
    prev_close = df['前收盘价']
    upper_limit = prev_close * 1.1
    lower_limit = prev_close * 0.9
    
    # 获取申报价格并判断是否触及限制
    declared_price = df['申报价格']
    is_at_upper = declared_price == upper_limit
    is_at_lower = declared_price == lower_limit
    
    # 有效约束条件：存在申报价格且不触及涨跌停价
    valid_constraint = has_declaration & ~(is_at_upper | is_at_lower)
    df['meu_18_7_constraint'] = valid_constraint

    return df

In [ ]:
df = check_meu_18_7(df)
df

---

# Law Article 19

## MEU_19_1


| 字段 | 内容 |
|------|------|
| subject | 上市公司 |
| condition | 实施竞价回购 |
| constraint | 实施期限不超过12个月 |
| contextual_info | 自董事会或股东大会（如须）审议通过回购股份决议之日起算 |
| note | nan |
| relation | nan |
| target | nan |
| type | 实际执行单元 |
| comments | nan |
| prompt_tokens | 1800 |
| completion_tokens | 4633 |


### 代码实现

In [ ]:
import pandas as pd

def check_meu_19_1(df):
    '''
    合规性检查函数，用于验证MEU_19_1条款的遵守情况：
    - subject: 上市公司（所有行自动满足）
    - condition: 实施竞价回购（需同时满足存在回购方案和回购方式为竞价）
    - constraint: 实施期限不超过12个月（基于决议通过日与实施截止日的时间差）
    '''
    df = df.copy()

    # 初始化标记列
    df['meu_19_1_subject'] = True  # 所有公司均为上市公司
    df['meu_19_1_condition'] = False
    df['meu_19_1_constraint'] = False

    # 标记条件：存在回购方案且回购方式为竞价
    is_auction_repurchase = (
        df['存在回购方案'] & 
        (df['回购方式'] == '竞价回购')
    )
    df.loc[is_auction_repurchase, 'meu_19_1_condition'] = True

    # 标记约束：实施期限不超过12个月
    # 仅当日期字段有效时进行计算
    valid_dates = df['决议通过日'].notna() & df['实施截止日'].notna()
    date_mask = valid_dates & (
        df['实施截止日'] <= df['决议通过日'] + pd.DateOffset(months=12)
    )
    df.loc[date_mask, 'meu_19_1_constraint'] = True

    return df

In [ ]:
df = check_meu_19_1(df)
df

---

## MEU_19_2


| 字段 | 内容 |
|------|------|
| subject | 上市公司 |
| condition | 因维护公司价值及股东权益所必需回购股份的，实施竞价回购 |
| constraint | 回购实施期限自股东大会或者董事会审议通过最终回购股份方案之日起不超过3个月 |
| contextual_info | nan |
| note | nan |
| relation | nan |
| target | nan |
| type | 实际执行单元 |
| comments | nan |
| prompt_tokens | 1810 |
| completion_tokens | 4004 |


### 代码实现

In [ ]:
import pandas as pd

def check_meu_19_2(df):
    '''
    检查MEU_19_2合规性：
    "subject": "上市公司", 
    "condition": "因维护公司价值及股东权益所必需回购股份的，实施竞价回购", 
    "constraint": "回购实施期限自股东大会或者董事会审议通过最终回购股份方案之日起不超过3个月", 
    "contextual_info": NaN
    '''
    df = df.copy()

    # 初始化合规标记列
    df['meu_19_2_subject'] = True  # 所有公司都是上市公司
    df['meu_19_2_condition'] = False
    df['meu_19_2_constraint'] = False

    # 1. 标记condition合规性
    # 条件需同时满足：存在回购方案、维护公司价值用途、竞价回购方式
    condition_mask = (
        df['存在回购方案'] &
        (df['回购用途'] == '维护公司价值及股东权益所必需') &
        (df['回购方式'] == '竞价回购')
    )
    df.loc[condition_mask, 'meu_19_2_condition'] = True

    # 2. 标记constraint合规性
    # 约束需同时满足：存在回购方案、实施截止日在决议通过日3个月内
    constraint_mask = df['存在回购方案'].copy()
    # 计算自然日边界（使用pd.DateOffset处理月份差异）
    deadline = df['决议通过日'] + pd.DateOffset(months=3)
    # 处理NaT避免类型错误（当存在回购方案为False时自动返回False）
    constraint_mask = constraint_mask & (df['实施截止日'] <= deadline)
    df.loc[constraint_mask, 'meu_19_2_constraint'] = True

    return df

In [ ]:
df = check_meu_19_2(df)
df

---

# Law Article 21

## MEU_21_2


| 字段 | 内容 |
|------|------|
| subject | 上市公司 |
| condition | 回购股份用于股权激励或者员工持股计划、转换上市公司发行的可转换为股票的公司债券、维护公司价值及股东权益所必需的 |
| constraint | 合计持有的本公司股份数不得超过本公司已发行股份总额的10% |
| contextual_info | nan |
| note | nan |
| relation | exclude |
| target | MEU_9_3 |
| type | 实际执行单元 |
| comments | nan |
| prompt_tokens | 1818 |
| completion_tokens | 2571 |


### 代码实现

In [ ]:
import pandas as pd

def check_meu_21_2(df):
    '''
    验证MEU_21_2合规性:
    "subject": "上市公司",
    "condition": "回购股份用于股权激励或者员工持股计划、转换上市公司发行的可转换为股票的公司债券、维护公司价值及股东权益所必需的",
    "constraint": "合计持有的本公司股份数不得超过本公司已发行股份总额的10%",
    "contextual_info": NaN
    '''

    df = df.copy()

    # 初始化标记列
    df['meu_21_2_subject'] = True  # 所有记录主体均为上市公司
    df['meu_21_2_condition'] = False
    df['meu_21_2_constraint'] = None

    # 1. 标记condition有效性
    valid_uses = [
        '股权激励或者员工持股计划',
        '转换上市公司发行的可转换为股票的公司债券',
        '维护公司价值及股东权益所必需'
    ]
    df['meu_21_2_condition'] = df['回购用途'].isin(valid_uses)

    # 2. 标记constraint有效性
    # 处理空值并计算回购比例
    total_shares = df['总股本'].replace(0, pd.NA)  # 防止除零错误
    cumulative_repo = df['累计回购数量'].fillna(0)
    repo_ratio = cumulative_repo / total_shares
    
    # 约束条件判断
    valid_constraint = (repo_ratio <= 0.1) & (~total_shares.isna())
    df['meu_21_2_constraint'] = valid_constraint.astype(bool)

    # 当总股本缺失时标记约束无效
    df.loc[total_shares.isna(), 'meu_21_2_constraint'] = False

    return df

In [ ]:
df = check_meu_21_2(df)
df

---

# Law Article 42

## MEU_42_2


| 字段 | 内容 |
|------|------|
| subject | 上市公司 |
| condition | 因维护公司价值及股东权益所必需而回购股份的，处于上市公司定期报告、业绩预告或者业绩快报披露前10个交易日内（因特殊原因推迟定期报告披露日期的，自原预约公告日前10个交易日起算，至披露前1个交易日） |
| constraint | 不得采用集中竞价交易方式出售股份 |
| contextual_info | nan |
| note | 简化假设: 不考虑报告推迟情况 |
| relation | nan |
| target | nan |
| type | 实际执行单元 |
| comments | nan |
| prompt_tokens | 1853 |
| completion_tokens | 9520 |


### 代码实现

In [ ]:
import pandas as pd

def check_meu_42_2(df):
    '''
    合规性检查函数，对应MEU_42_2条款：
    - subject: 上市公司
    - condition: 因维护公司价值及股东权益回购股份且处于定期报告披露前10个交易日内
    - constraint: 不得采用集中竞价交易方式出售股份
    '''
    df = df.copy()

    # 初始化标记列
    df['meu_42_2_subject'] = True  # 所有公司均为上市公司
    df['meu_42_2_condition'] = False
    df['meu_42_2_constraint'] = False

    # 1. 计算时间窗口条件（condition的时间部分）
    # 获取所有定期报告类公告日期
    report_mask = df['公告类型'].isin(['定期报告', '业绩预告', '业绩快报'])
    valid_announcements = df[report_mask][['公司简称', '公告日期']].drop_duplicates()
    
    # 计算时间窗口标记列
    df['_temp_in_window'] = False
    if not valid_announcements.empty:
        # 生成时间窗口范围（自然日简化计算）
        valid_announcements['start_date'] = valid_announcements['公告日期'] - pd.DateOffset(days=10)
        
        # 合并并标记有效窗口
        merged = df.merge(valid_announcements, on='公司简称', suffixes=('', '_ann'))
        time_condition = (merged['日期'] >= merged['start_date']) & (merged['日期'] < merged['公告日期_ann'])
        window_idx = merged[time_condition].index.unique()
        df.loc[window_idx, '_temp_in_window'] = True

    # 2. 标记condition条件
    usage_condition = df['回购用途'] == '维护公司价值及股东权益所必需'
    df['meu_42_2_condition'] = usage_condition & df['_temp_in_window']

    # 3. 标记constraint条件（独立检查）
    # 集中竞价出售包含计划申报和实际成交两种情形
    sell_condition = df['存在出售计划'] | (df['当日出售数量'] > 0)
    df['meu_42_2_constraint'] = ~sell_condition

    # 清理临时列
    df.drop(columns=['_temp_in_window'], inplace=True, errors='ignore')

    return df

In [ ]:
df = check_meu_42_2(df)
df

---

# Law Article 45

## MEU_45_2


| 字段 | 内容 |
|------|------|
| subject | 上市公司 |
| condition | 采用集中竞价交易方式出售已回购股份 |
| constraint | 申报价格不得为公司股票当日交易跌幅限制的价格 |
| contextual_info | nan |
| note | nan |
| relation | nan |
| target | nan |
| type | 实际执行单元 |
| comments | nan |
| prompt_tokens | 1796 |
| completion_tokens | 8055 |


### 代码实现

In [ ]:
import pandas as pd

def check_meu_45_2(df):
    '''
    检查MEU_45_2的合规性：
    subject: 上市公司（所有行均为上市公司）
    condition: 采用集中竞价交易方式出售已回购股份（存在出售计划即视为满足）
    constraint: 申报价格≠当日跌停价（前收盘价*0.9）
    '''
    df = df.copy()

    # 初始化标记列
    df['meu_45_2_subject'] = True  # 所有公司均为上市公司
    df['meu_45_2_condition'] = df['存在出售计划']  # 直接映射存在出售计划字段
    df['meu_45_2_constraint'] = None  # 初始化为None以便后续布尔赋值

    # 计算当日跌停价（前收盘价的90%）
    df['_temp_lower_limit'] = df['前收盘价'] * 0.9
    
    # 构建约束有效性判断条件
    price_condition = df['申报价格'] != df['_temp_lower_limit']
    na_condition = df['申报价格'].isna()
    valid_constraint = price_condition | na_condition  # 申报价≠跌停价或未申报均视为合规

    # 标记约束有效性
    df.loc[valid_constraint, 'meu_45_2_constraint'] = True
    df.loc[~valid_constraint, 'meu_45_2_constraint'] = False

    # 清理临时列
    df.drop(columns=['_temp_lower_limit'], inplace=True)

    return df

In [ ]:
df = check_meu_45_2(df)
df

---

## MEU_45_3


| 字段 | 内容 |
|------|------|
| subject | 上市公司 |
| condition | 采用集中竞价交易方式出售已回购股份且每日出售数量超过10万股 |
| constraint | 每日出售的数量不得超过出售计划披露日前20个交易日日均盘中成交量的25% |
| contextual_info | nan |
| note | nan |
| relation | nan |
| target | nan |
| type | 实际执行单元 |
| comments | nan |
| prompt_tokens | 1810 |
| completion_tokens | 8673 |


### 代码实现

In [ ]:
import pandas as pd

def check_meu_45_3(df):
    '''
    检查MEU_45_3合规性：
    subject: 上市公司
    condition: 采用集中竞价交易方式出售已回购股份且每日出售数量超过10万股
    constraint: 每日出售数量不得超过出售计划披露日前20个交易日日均盘中成交量的25%
    '''
    df = df.copy()

    # 初始化标记列
    df['meu_45_3_subject'] = True  # 所有记录均为上市公司
    df['meu_45_3_condition'] = False
    df['meu_45_3_constraint'] = False  # 默认不满足约束

    # 1. 标记condition条件
    # 条件需同时满足：存在出售计划、采用集中竞价方式（隐含在存在出售计划中）、当日出售数量>10万
    valid_condition = df['存在出售计划'] & (df['当日出售数量'] > 100000)
    df.loc[valid_condition, 'meu_45_3_condition'] = True

    # 2. 标记constraint约束（独立检查所有记录）
    # 预处理：计算每个公司每个披露日前20日成交量均值
    # 假设存在'成交量'列且'公告日期'为计划披露日
    # 按公司分组处理
    for company in df['公司简称'].unique():
        company_df = df[df['公司简称'] == company].sort_values('日期')
        
        # 创建日期索引
        company_dates = company_df.set_index('日期')
        
        # 遍历该公司的所有记录
        for idx, row in company_df.iterrows():
            disclosure_date = row['公告日期']
            if pd.isnull(disclosure_date):
                continue
            
            # 获取披露日前20个交易日的成交量
            try:
                # 筛选披露日前的交易日
                mask = company_dates.index < disclosure_date
                prior_sessions = company_dates[mask].last('20D')
                
                if len(prior_sessions) >= 20:
                    avg_volume = prior_sessions['成交量'].mean()
                    threshold = avg_volume * 0.25
                    current_sell = row['当日出售数量']
                    
                    # 标记约束满足情况
                    if current_sell <= threshold:
                        df.loc[idx, 'meu_45_3_constraint'] = True
            except KeyError:
                # 日期索引不存在时跳过
                continue

    return df

In [ ]:
df = check_meu_45_3(df)
df

---

## MEU_45_4


| 字段 | 内容 |
|------|------|
| subject | 上市公司 |
| condition | 采用集中竞价交易方式出售已回购股份 |
| constraint | 在任意连续90日内，出售股份的总数不得超过公司股份总数的1% |
| contextual_info | nan |
| note | nan |
| relation | nan |
| target | nan |
| type | 实际执行单元 |
| comments | nan |
| prompt_tokens | 1802 |
| completion_tokens | 5301 |


### 代码实现

In [ ]:
import pandas as pd

def check_meu_45_4(df):
    '''
    检查MEU_45_4合规性：
    - subject: 上市公司（所有行自动满足）
    - condition: 存在集中竞价出售已回购股份计划
    - constraint: 90交易日累计出售量≤总股本1%
    '''
    df = df.copy()

    # 初始化合规标记列
    df['meu_45_4_subject'] = True  # 所有公司均为上市公司
    df['meu_45_4_condition'] = df['存在出售计划'].astype(bool)  # 存在出售计划即满足条件
    df['meu_45_4_constraint'] = None  # 初始化为空值

    # 按公司分组计算滚动累计出售量
    df.sort_values(['公司简称', '日期'], inplace=True)
    df['90交易日累计出售'] = df.groupby('公司简称')['当日出售数量'].transform(
        lambda x: x.rolling(90, min_periods=1).sum()
    )

    # 计算总股本1%阈值并验证约束
    threshold = df['总股本'] * 0.01
    df['meu_45_4_constraint'] = df['90交易日累计出售'] <= threshold

    # 清理中间列
    df.drop(columns=['90交易日累计出售'], inplace=True)

    return df

In [ ]:
df = check_meu_45_4(df)
df

---

# Law Article 46

## MEU_46_2


| 字段 | 内容 |
|------|------|
| subject | 上市公司 |
| condition | 采用集中竞价交易方式出售已回购股份期间且首次出售已回购股份事实发生后 |
| constraint | 应当在次1个交易日内披露出售进展情况公告 |
| contextual_info | nan |
| note | nan |
| relation | nan |
| target | nan |
| type | 实际执行单元 |
| comments | nan |
| prompt_tokens | 1806 |
| completion_tokens | 9458 |


### 代码实现

In [ ]:
import pandas as pd

def check_meu_46_2(df):
    '''
    检查MEU_46_2合规性：
    "subject": "上市公司", 
    "condition": "采用集中竞价交易方式出售已回购股份期间且首次出售已回购股份事实发生后", 
    "constraint": "应当在次1个交易日内披露出售进展情况公告"
    '''
    df = df.copy()
    
    # 初始化标记列
    df['meu_46_2_subject'] = True  # 所有公司都是上市公司
    df['meu_46_2_condition'] = False
    df['meu_46_2_constraint'] = False
    
    # 条件1：标记subject（所有行自动满足）
    
    # 条件2：标记condition
    # 步骤1：筛选存在集中竞价出售计划的行
    mask_condition = df['存在出售计划'] == True
    
    # 步骤2：判断是否在出售期间内
    mask_date_range = (df['日期'] >= df['出售开始日']) & (df['日期'] <= df['出售截止日'])
    
    # 步骤3：计算首次出售日期（按公司+出售计划分组）
    group_cols = ['公司简称', '出售开始日', '出售截止日']
    df['首次出售日期'] = df.groupby(group_cols)['日期'].transform(
        lambda g: g[g['累计出售数量'] > 0]['日期'].min() if any(g['累计出售数量'] > 0) else pd.NaT
    )
    
    # 步骤4：判断当前日期是否在首次出售之后
    mask_first_sale = (df['日期'] >= df['首次出售日期']) & (~df['首次出售日期'].isna())
    
    # 合并condition条件
    valid_condition = mask_condition & mask_date_range & mask_first_sale
    df.loc[valid_condition, 'meu_46_2_condition'] = True
    
    # 条件3：标记constraint
    # 步骤1：获取每个首次出售的次交易日
    df_sorted = df.sort_values(['公司简称', '日期'])
    df_sorted['次交易日'] = df_sorted.groupby('公司简称')['日期'].shift(-1)
    
    # 步骤2：建立首次出售与次日期的映射关系
    first_sales = df[valid_condition].groupby(group_cols)['首次出售日期'].first().reset_index()
    first_sales = first_sales.merge(
        df_sorted[['公司简称', '日期', '次交易日']],
        left_on=['公司简称', '首次出售日期'],
        right_on=['公司简称', '日期'],
        how='left'
    )
    first_sales_map = first_sales[group_cols + ['次交易日']]
    
    # 步骤3：合并到原始数据
    df = df.merge(first_sales_map, on=group_cols, how='left')
    
    # 步骤4：验证次日披露情况
    valid_constraint = (df['日期'] == df['次交易日']) & (df['披露出售进展'] == True)
    df.loc[valid_constraint, 'meu_46_2_constraint'] = True
    
    # 清理临时列
    df.drop(columns=['首次出售日期', '次交易日'], inplace=True, errors='ignore')
    
    return df

In [ ]:
df = check_meu_46_2(df)
df

---

## MEU_46_3


| 字段 | 内容 |
|------|------|
| subject | 上市公司 |
| condition | 采用集中竞价交易方式出售已回购股份期间且出售已回购股份占上市公司总股本的比例增加1% |
| constraint | 应当在事实发生之日起3个交易日内披露出售进展情况公告 |
| contextual_info | nan |
| note | nan |
| relation | nan |
| target | nan |
| type | 实际执行单元 |
| comments | nan |
| prompt_tokens | 1813 |
| completion_tokens | 9654 |


### 代码实现

In [ ]:
import pandas as pd

def check_meu_46_3(df):
    '''
    检查MEU_46_3合规性：
    subject: 上市公司（所有行自动满足）
    condition: 1.处于集中竞价出售期间 2.当日累计出售比例较前日增加≥1%
    constraint: 在触发条件的3个交易日内披露进展
    '''
    df = df.copy()
    
    # 初始化标记列
    df['meu_46_3_subject'] = True  # 所有公司均为上市公司
    df['meu_46_3_condition'] = False
    df['meu_46_3_constraint'] = False
    
    # 条件1：存在有效的出售计划（假设均为集中竞价方式）
    in_selling_period = df['存在出售计划']
    
    # 条件2：计算累计出售比例日增量
    df['累计出售比例_prev'] = df.groupby('公司简称')['累计出售比例'].shift(1).fillna(0)
    threshold_met = (df['累计出售比例'] - df['累计出售比例_prev']) >= 0.01
    
    # 标记满足条件的行
    valid_condition = in_selling_period & threshold_met
    df.loc[valid_condition, 'meu_46_3_condition'] = True
    
    # 约束检查：仅对满足条件的行检查后续3日披露
    announcement_dates = df[df['披露出售进展']][['公司简称', '日期']]
    
    # 使用merge_asof查找最近3日内的公告
    if not valid_condition.empty and not announcement_dates.empty:
        merged = pd.merge_asof(
            df[valid_condition][['公司简称', '日期']].sort_values('日期'),
            announcement_dates.sort_values('日期'),
            on='日期',
            by='公司简称',
            direction='forward',
            tolerance=pd.Timedelta(days=3)
        )
        # 创建约束满足标记
        constraint_mask = merged['日期_y'].notna()
        # 将结果映射回原dataframe
        df.loc[valid_condition, 'meu_46_3_constraint'] = (
            constraint_mask.values
            if len(constraint_mask) == valid_condition.sum() 
            else False
        )
    
    # 清理中间列
    df.drop(columns=['累计出售比例_prev'], inplace=True, errors='ignore')
    
    return df

In [ ]:
df = check_meu_46_3(df)
df

---

# Law Article 50

## MEU_50_2


| 字段 | 内容 |
|------|------|
| subject | 上市公司 |
| condition | 实施要约回购 |
| constraint | 要约期限不得少于30个自然日 |
| contextual_info | nan |
| note | nan |
| relation | nan |
| target | nan |
| type | 实际执行单元 |
| comments | nan |
| prompt_tokens | 1788 |
| completion_tokens | 3462 |


### 代码实现

In [ ]:
import pandas as pd

def check_meu_50_2(df):
    '''
    检查MEU_50_2合规性：
    "subject": "上市公司",
    "condition": "实施要约回购",
    "constraint": "要约期限不得少于30个自然日",
    "contextual_info": NaN
    '''
    df = df.copy()
    
    # 初始化合规标记列
    df['meu_50_2_subject'] = True  # 所有公司均为上市公司
    df['meu_50_2_condition'] = False
    df['meu_50_2_constraint'] = False
    
    # 标记valid的condition（存在回购方案且为要约回购）
    is_repurchase = df['存在回购方案']
    is_tender_offer = df['回购方式'] == '要约回购'
    df.loc[is_repurchase & is_tender_offer, 'meu_50_2_condition'] = True
    
    # 标记valid的constraint（自然日计算要约期限）
    # 处理NaN并转换为布尔值
    valid_constraint = (df['要约期限'] >= 30).fillna(False)
    df['meu_50_2_constraint'] = valid_constraint.astype(bool)
    
    return df

In [ ]:
df = check_meu_50_2(df)
df

---

## MEU_50_3


| 字段 | 内容 |
|------|------|
| subject | 上市公司 |
| condition | 实施要约回购 |
| constraint | 要约期限不得超过60个自然日 |
| contextual_info | nan |
| note | nan |
| relation | nan |
| target | nan |
| type | 实际执行单元 |
| comments | nan |
| prompt_tokens | 1787 |
| completion_tokens | 3882 |


### 代码实现

In [ ]:
import pandas as pd

def check_meu_50_3(df):
    '''
    合规性检查函数，对应MEU_50_3条款：
    - subject: 上市公司
    - condition: 实施要约回购
    - constraint: 要约期限不得超过60个自然日
    '''
    df = df.copy()

    # 初始化标记列
    df['meu_50_3_subject'] = True  # 所有公司均为上市公司
    df['meu_50_3_condition'] = False
    df['meu_50_3_constraint'] = False

    # 标记条件：实施要约回购（存在有效回购方案且回购方式为要约回购）
    valid_condition = (df['存在回购方案']) & (df['回购方式'] == '要约回购')
    df.loc[valid_condition, 'meu_50_3_condition'] = True

    # 标记约束：计算自然日差（需处理空值）
    has_valid_dates = df['实施开始日'].notna() & df['实施截止日'].notna()
    duration = (df['实施截止日'] - df['实施开始日']).dt.days
    valid_constraint = (duration <= 60) & has_valid_dates
    
    # 更新约束标记（独立检查，不依赖subject和condition）
    df.loc[valid_constraint, 'meu_50_3_constraint'] = True

    return df

In [ ]:
df = check_meu_50_3(df)
df

---

# Law Article 51

## MEU_51_2


| 字段 | 内容 |
|------|------|
| subject | 上市公司 |
| condition | 实施要约回购 |
| constraint | 要约价格不得低于回购股份方案公告日前30个交易日该种股票每日加权平均价的算术平均值 |
| contextual_info | nan |
| note | nan |
| relation | refer_to |
| target | Law_75 |
| type | 实际执行单元 |
| comments | nan |
| prompt_tokens | 1865 |
| completion_tokens | 6265 |


### 代码实现

In [ ]:
import pandas as pd

def check_meu_51_2(df):
    '''
    验证MEU_51_2合规性：
    - subject: 上市公司（所有行自动满足）
    - condition: 实施要约回购（回购方式为要约回购）
    - constraint: 要约价格不低于方案公告日前30个交易日平均价
    '''
    df = df.copy()

    # 初始化标记列
    df['meu_51_2_subject'] = True  # 所有公司均为上市公司
    df['meu_51_2_condition'] = df['回购方式'] == '要约回购'
    df['meu_51_2_constraint'] = False  # 默认设为False

    # 计算前30日交易均价逻辑
    # 步骤1：提取有效决议记录
    valid_resolutions = df[['公司简称', '决议通过日']].dropna().drop_duplicates()
    
    # 步骤2：计算每个决议对应的30日均价
    avg_price_list = []
    for _, row in valid_resolutions.iterrows():
        company = row['公司简称']
        resolution_date = row['决议通过日']
        
        # 获取决议日前30个交易日数据
        company_data = df[(df['公司简称'] == company) & (df['日期'] < resolution_date)]
        sorted_dates = company_data.sort_values('日期', ascending=False)
        last_30 = sorted_dates.head(30)
        
        if len(last_30) == 30:
            avg_price = last_30['收盘价'].mean()
            avg_price_list.append({
                '公司简称': company,
                '决议通过日': resolution_date,
                'ref_price': avg_price
            })

    # 步骤3：合并参考价格到主表
    if avg_price_list:
        ref_df = pd.DataFrame(avg_price_list)
        df = df.merge(ref_df, on=['公司简称', '决议通过日'], how='left')
        
        # 步骤4：执行价格比较（仅当要约价格和参考价均存在时）
        price_mask = df['要约价格'].notna() & df['ref_price'].notna()
        df.loc[price_mask, 'meu_51_2_constraint'] = df.loc[price_mask, '要约价格'] >= df.loc[price_mask, 'ref_price']
        
        # 清理临时列
        df.drop(columns=['ref_price'], inplace=True)

    return df

In [ ]:
df = check_meu_51_2(df)
df

---